In [12]:
import sys
from Bio import SeqIO
 
def main():
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")
    input = sys.stdin.readline
    
    for record in SeqIO.parse(sys.stdin, "fasta"):
        print(len(record.seq))    
main()

In [ ]:
# Parses fasta file
def read_fasta_file(inp_fasta_file):
    name, seq = '', []
    for line in inp_fasta_file:
        line = line.rstrip()
        if line.startswith(">"):
            if name:
                yield(name, ''.join(seq))
            name, seq = line, []
        else: 
            seq.append(line)
    if name: 
        yield(name, ''.join(seq))

In [48]:
import sys
from Bio import SeqIO

def score(x, y):
    if x == y:
        return 1
    else:
        return -1

def main():
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")

    sequences = [record.seq for record in SeqIO.parse(sys.stdin, "fasta")]
    seq1, seq2 = sequences[0], sequences[1]

    M = [[0] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    
    for i in range(1, len(seq1) + 1):
        M[i][0] = -i
    for j in range(1, len(seq2) + 1):
        M[0][j] = -j
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            M[i][j] = max(M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1]), \
                          M[i - 1][j] + score(seq1[i - 1], '-'), \
                          M[i][j - 1] + score('-', seq2[j - 1]))
    print(M[-1][-1])
main()

In [6]:
import sys
from Bio import SeqIO

def score(x, y):
    if x == y:
        return 1
    else:
        return -1

def main():
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")

    sequences = [record.seq for record in SeqIO.parse(sys.stdin, "fasta")]
    seq1, seq2 = sequences[0], sequences[1]

    M = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    p = [[None] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    
    M[0][0] = 0
    for i in range(1, len(seq1) + 1):
        M[i][0] = -i
    for j in range(1, len(seq2) + 1):
        M[0][j] = -j
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1]) > M[i][j]:
                M[i][j] = M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1])
                p[i][j] = (-1, -1)
            if M[i - 1][j] + score(seq1[i - 1], '-') > M[i][j]:
                M[i][j] = M[i - 1][j] + score(seq1[i - 1], '-')
                p[i][j] = (-1, 0)
            if M[i][j - 1] + score('-', seq2[j - 1]) > M[i][j]:
                M[i][j] = M[i][j - 1] + score('-', seq2[j - 1])
                p[i][j] = (0, -1)
    print(M[-1][-1])
    
    res1, res2 = [], []
    i, j = len(seq1), len(seq2)
    while p[i][j] != None:
        if p[i][j] == (-1, 0):
            res1.append(seq1[i - 1])
            res2.append('-')
            i -= 1
        elif p[i][j] == (0, -1):
            res1.append('-')
            res2.append(seq2[j - 1])
            j -= 1
        elif p[i][j] == (-1, -1):
            if seq1[i - 1] != seq2[j - 1]:
                res1.append(seq1[i - 1].lower())
                res2.append(seq2[j - 1].lower())
            else:
                res1.append(seq1[i - 1])
                res2.append(seq2[j - 1])
            i, j = i - 1, j -1
                
    res1.reverse()
    res2.reverse()
    
    print(''.join(res1))
    print(''.join(res2))
main()

In [23]:
import sys
from Bio import SeqIO

def score(x, y):
    if x == y:
        return 1
    else:
        return -1

M, p = [], []
seq1, seq2 = '', ''
res1, res2 = [], []

def possible_paths(i, j):
    if i == 0 and j == 0:
        tmp1, tmp2 = res1.copy(), res2.copy()
        tmp1.reverse()
        tmp2.reverse()
        print(''.join(tmp1))
        print(''.join(tmp2))
        return
    
    for path in p[i][j]:
        if path == (-1, 0):
            res1.append(seq1[i - 1])
            res2.append('-')
            possible_paths(i - 1, j)
        elif path == (0, -1):
            res1.append('-')
            res2.append(seq2[j - 1])
            possible_paths(i, j - 1)
        elif path == (-1, -1):
            if seq1[i - 1] != seq2[j - 1]:
                res1.append(seq1[i - 1].lower())
                res2.append(seq2[j - 1].lower())
            else:
                res1.append(seq1[i - 1])
                res2.append(seq2[j - 1])
            possible_paths(i - 1, j - 1)
        res1.pop()
        res2.pop()
    

def main():
    global M, p, seq1, seq2
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")

    sequences = [record.seq for record in SeqIO.parse(sys.stdin, "fasta")]
    seq1, seq2 = sequences[0], sequences[1]

    M = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    p = [[[] for j in range(len(seq2) + 1)] for i in range(len(seq1) + 1)] 
    
    M[0][0] = 0
    for i in range(1, len(seq1) + 1):
        M[i][0] = -i
    for j in range(1, len(seq2) + 1):
        M[0][j] = -j
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1]) >= M[i][j]:
                if M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1]) > M[i][j]:
                    p[i][j].clear()
                M[i][j] = M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1])
                p[i][j].append((-1, -1))
            if M[i - 1][j] + score(seq1[i - 1], '-') >= M[i][j]:
                if M[i - 1][j] + score(seq1[i - 1], '-') > M[i][j]:
                    p[i][j].clear()
                M[i][j] = M[i - 1][j] + score(seq1[i - 1], '-')
                p[i][j].append((-1, 0))
            if M[i][j - 1] + score('-', seq2[j - 1]) >= M[i][j]:
                if M[i][j - 1] + score('-', seq2[j - 1]) > M[i][j]:
                    p[i][j].clear()
                M[i][j] = M[i][j - 1] + score('-', seq2[j - 1])
                p[i][j].append((0, -1))
                
    print(M[-1][-1])
    possible_paths(len(seq1), len(seq2))
main()

In [77]:
import sys
from Bio import SeqIO

def score(x, y):
    if x == y:
        return 1
    else:
        return -1
    
max_score = 0
min_i, max_i = 0, 0
min_j, max_j = 0, 0

def smith_waterman(seq1, seq2):
    global max_score
    global min_i, max_i, min_j, max_j
    M = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    p = [[None] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    
    M[0][0] = 0
    for i in range(1, len(seq1) + 1):
        M[i][0] = 0
    for j in range(1, len(seq2) + 1):
        M[0][j] = 0
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            M[i][j] = 0
            if M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1]) > M[i][j]:
                M[i][j] = M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1])
                p[i][j] = (-1, -1)
            if M[i - 1][j] + score(seq1[i - 1], '-') > M[i][j]:
                M[i][j] = M[i - 1][j] + score(seq1[i - 1], '-')
                p[i][j] = (-1, 0)
            if M[i][j - 1] + score('-', seq2[j - 1]) > M[i][j]:
                M[i][j] = M[i][j - 1] + score('-', seq2[j - 1])
                p[i][j] = (0, -1)
    
    max_i, max_j = 0, 0
    max_score = max(map(max, M))
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if M[i][j] == max_score:
                max_i, max_j = i, j
                break
    
    res1, res2 = [], []
    i, j = max_i, max_j
    while p[i][j] != None:
        if p[i][j] == (-1, 0):
            res1.append(seq1[i - 1])
            res2.append('-')
            i -= 1
        elif p[i][j] == (0, -1):
            res1.append('-')
            res2.append(seq2[j - 1])
            j -= 1
        elif p[i][j] == (-1, -1):
            if seq1[i - 1] != seq2[j - 1]:
                res1.append(seq1[i - 1].lower())
                res2.append(seq2[j - 1].lower())
            else:
                res1.append(seq1[i - 1])
                res2.append(seq2[j - 1])
            i, j = i - 1, j -1
        min_i, min_j = i, j           
    res1.reverse()
    res2.reverse()
    return ''.join(res1), ''.join(res2)

def needleman_wunsch(seq1, seq2):
    M = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    p = [[None] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    
    M[0][0] = 0
    for i in range(1, len(seq1) + 1):
        M[i][0] = -i
        p[i][0] = (-1, 0)
    for j in range(1, len(seq2) + 1):
        M[0][j] = -j
        p[0][j] = (0, -1)
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1]) > M[i][j]:
                M[i][j] = M[i - 1][j - 1] + score(seq1[i - 1], seq2[j - 1])
                p[i][j] = (-1, -1)
            if M[i - 1][j] + score(seq1[i - 1], '-') > M[i][j]:
                M[i][j] = M[i - 1][j] + score(seq1[i - 1], '-')
                p[i][j] = (-1, 0)
            if M[i][j - 1] + score('-', seq2[j - 1]) > M[i][j]:
                M[i][j] = M[i][j - 1] + score('-', seq2[j - 1])
                p[i][j] = (0, -1)
    
    res1, res2 = [], []
    i, j = len(seq1), len(seq2)
    while p[i][j] != None:
        if p[i][j] == (-1, 0):
            res1.append(seq1[i - 1])
            res2.append('-')
            i -= 1
        elif p[i][j] == (0, -1):
            res1.append('-')
            res2.append(seq2[j - 1])
            j -= 1
        elif p[i][j] == (-1, -1):
            if seq1[i - 1] != seq2[j - 1]:
                res1.append(seq1[i - 1].lower())
                res2.append(seq2[j - 1].lower())
            else:
                res1.append(seq1[i - 1])
                res2.append(seq2[j - 1])
            i, j = i - 1, j -1 
    res1.reverse()
    res2.reverse()
    return ''.join(res1), ''.join(res2)
    
    
def main():
    global max_score
    global min_i, max_i, min_j, max_j
    
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")

    sequences = [record.seq for record in SeqIO.parse(sys.stdin, "fasta")]
    seq1, seq2 = sequences[0], sequences[1]    
    
    seq1_middle, seq2_middle = smith_waterman(seq1, seq2)
    print(max_score)
    
    seq1_left, seq2_left = needleman_wunsch(seq1[:min_i], seq2[:min_j])
    seq1_right, seq2_right = needleman_wunsch(seq1[max_i:], seq2[max_j:])
    
    print(seq1_left.lower() + seq1_middle + seq1_right.lower())
    print(seq2_left.lower() + seq2_middle + seq2_right.lower())
main()

In [95]:
ordinal = {'A': 0, 'C': 1, 'G': 2, 'T': 3}

def needleman_wunsch(d, weights, seq1, seq2):
    M = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    p = [[None] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    
    M[0][0] = 0
    for i in range(1, len(seq1) + 1):
        M[i][0] = -i
        p[i][0] = (-1, 0)
    for j in range(1, len(seq2) + 1):
        M[0][j] = -j
        p[0][j] = (0, -1)
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if M[i - 1][j - 1] + weights[ordinal[seq1[i - 1]]][ordinal[seq2[j - 1]]] > M[i][j]:
                M[i][j] = M[i - 1][j - 1] + weights[ordinal[seq1[i - 1]]][ordinal[seq2[j - 1]]]
                p[i][j] = (-1, -1)
            if M[i - 1][j] + d > M[i][j]:
                M[i][j] = M[i - 1][j] + d
                p[i][j] = (-1, 0)
            if M[i][j - 1] + d > M[i][j]:
                M[i][j] = M[i][j - 1] + d
                p[i][j] = (0, -1)
    print(M[-1][-1])
    
    res1, res2 = [], []
    i, j = len(seq1), len(seq2)
    while p[i][j] != None:
        if p[i][j] == (-1, 0):
            res1.append(seq1[i - 1])
            res2.append('-')
            i -= 1
        elif p[i][j] == (0, -1):
            res1.append('-')
            res2.append(seq2[j - 1])
            j -= 1
        elif p[i][j] == (-1, -1):
            if seq1[i - 1] != seq2[j - 1]:
                res1.append(seq1[i - 1].lower())
                res2.append(seq2[j - 1].lower())
            else:
                res1.append(seq1[i - 1])
                res2.append(seq2[j - 1])
            i, j = i - 1, j -1 
    res1.reverse()
    res2.reverse()
    print(''.join(res1))
    print(''.join(res2))
    return ''.join(res1), ''.join(res2)

import sys
from Bio import SeqIO

def main():
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")
    input = sys.stdin.readline

    d = int(input())
    
    weights = [[0] * 4 for i in range(4)]
    for i in range(4):
        j = 0
        for weight in map(int, input().split()):
            weights[i][j] = weight
            j += 1
            
    sequences = [record.seq for record in SeqIO.parse(sys.stdin, "fasta")]
    seq1, seq2 = sequences[0], sequences[1]
    
    needleman_wunsch(d, weights, seq1, seq2)
main()

In [140]:
def needleman_wunsch(d_same, d_replace, d_open, d_extension, seq1, seq2):
    M = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    D = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    p = [[None] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    
    M[0][0] = 0
    D[0][0] = 0
    for i in range(1, len(seq1) + 1):
        #M[i][0] = -i
        #M[i][0] = 0
        D[i][0] = d_open + (i - 1) * d_extension
        p[i][0] = (-1, 0)
    for j in range(1, len(seq2) + 1):
        #M[0][j] = -j
        #M[0][j] = 0
        D[0][j] = d_open + (j - 1) * d_extension
        p[0][j] = (0, -1)
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            
            #D[i][j] = max(M[i - 1][j] + d_open, M[i][j - 1] + d_open, D[i - 1][j] + d_extension, D[i][j - 1] + d_extension)
            
            D[i][j] = max(M[i - 1][j] + d_open, D[i - 1][j] + d_extension, M[i][j - 1] + d_open, D[i][j - 1] + d_extension)
#             if M[i - 1][j] + d_open > D[i][j] or D[i - 1][j] + d_extension > D[i][j]:
#                 D[i][j] = max(M[i - 1][j] + d_open, D[i - 1][j] + d_extension)
#                 p[i][j] = (-1, 0)
#             if M[i][j - 1] + d_open > D[i][j] or D[i][j - 1] + d_extension > D[i][j]:
#                 D[i][j] = max(M[i][j - 1] + d_open, D[i][j - 1] + d_extension)
#                 p[i][j] = (0, -1)
                
                
            if seq1[i - 1] == seq2[j - 1]:
                d = d_same
            else: 
                d = d_replace
                
            M[i][j] = max(M[i - 1][j - 1] + d, D[i - 1][j - 1] + d)  
                
#             if M[i - 1][j - 1] + d > M[i][j]:
#                 M[i][j] = M[i - 1][j - 1] + d
#                 p[i][j] = (-1, -1)
#             if D[i - 1][j - 1] + d > M[i][j]:
#                 M[i][j] = D[i - 1][j - 1] + d
#                 p[i][j] = (-1, -1)
    print(M[-1][-1])
    for i in range(len(seq1) + 1):
        print(M[i])
    print()
    for i in range(len(seq1) + 1):
        print(D[i])
          
    res1, res2 = [], []
    i, j = len(seq1), len(seq2)
    while i != 0 or j != 0:
        tmp = max(M[i - 1][j], D[i - 1][j])
        p[i][j] = (-1, 0)
        if max(M[i][j - 1], D[i][j - 1]) > tmp:
            tmp = max(M[i][j - 1], D[i][j - 1])
            p[i][j] = (0, -1)
        if max(M[i - 1][j - 1], D[i - 1][j - 1]) > tmp:
            tmp = max(M[i - 1][j - 1], D[i - 1][j - 1])
            p[i][j] = (-1, -1)
        
        if p[i][j] == (-1, 0):
            res1.append(seq1[i - 1])
            res2.append('-')
            i -= 1
        elif p[i][j] == (0, -1):
            res1.append('-')
            res2.append(seq2[j - 1])
            j -= 1
        elif p[i][j] == (-1, -1):
            if seq1[i - 1] != seq2[j - 1]:
                res1.append(seq1[i - 1].lower())
                res2.append(seq2[j - 1].lower())
            else:
                res1.append(seq1[i - 1])
                res2.append(seq2[j - 1])
            i, j = i - 1, j -1 
    res1.reverse()
    res2.reverse()
    print(''.join(res1))
    print(''.join(res2))
#     for i in range(len(seq1)):
#         print(p[i])
    return ''.join(res1), ''.join(res2)
            
import sys
from Bio import SeqIO

def main():
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")
    input = sys.stdin.readline

    d_same, d_replace, d_open, d_extension = list(map(float, input().split()))
            
    sequences = [record.seq for record in SeqIO.parse(sys.stdin, "fasta")]
    seq1, seq2 = sequences[0], sequences[1]
    
    needleman_wunsch(d_same, d_replace, d_open, d_extension, seq1, seq2)
main()

In [149]:
def needleman_wunsch(d_same, d_replace, d_open, d_extension, seq1, seq2):
    M = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    Del = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    Ins = [[float('-inf')] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    p = [[None] * (len(seq2) + 1) for i in range(len(seq1) + 1)] 
    
    M[0][0] = 0
    Del[0][0] = 0
    Ins[0][0] = 0
    for i in range(1, len(seq1) + 1):
        Del[i][0] = d_open + (i - 1) * d_extension
        p[i][0] = (-1, 0)
    for j in range(1, len(seq2) + 1):
        Ins[0][j] = d_open + (j - 1) * d_extension
        p[0][j] = (0, -1)
        
    for i in range(1, len(seq1) + 1):
        for j in range(1, len(seq2) + 1):
            if seq1[i - 1] == seq2[j - 1]:
                d = d_same
            else: 
                d = d_replace
            Del[i][j] = max(M[i - 1][j] + d_open, Ins[i - 1][j] + d_open, Del[i - 1][j] + d_extension)
            Ins[i][j] = max(M[i][j - 1] + d_open, Del[i][j - 1] + d_open, Ins[i][j - 1] + d_extension)
            M[i][j] = max(M[i - 1][j - 1] + d, Del[i - 1][j - 1] + d, Ins[i - 1][j - 1] + d)  
                
    print(max(M[-1][-1], Del[-1][-1], Ins[-1][-1]))
#     for i in range(len(seq1) + 1):
#         print(M[i])
#     print()
#     for i in range(len(seq1) + 1):
#         print(Del[i])
#     print()
#     for i in range(len(seq1) + 1):
#         print(Ins[i])  
          
    res1, res2 = [], []
    i, j = len(seq1), len(seq2)
    while i != 0 or j != 0:
        tmp = max(M[i - 1][j], Del[i - 1][j], Ins[i - 1][j])
        p[i][j] = (-1, 0)
        if max(M[i][j - 1], Del[i][j - 1], Ins[i][j - 1]) > tmp:
            tmp = max(M[i][j - 1], Del[i][j - 1], Ins[i][j - 1])
            p[i][j] = (0, -1)
        if max(M[i - 1][j - 1], Del[i - 1][j - 1], Ins[i - 1][j - 1]) > tmp:
            tmp = max(M[i - 1][j - 1], Del[i - 1][j - 1], Ins[i - 1][j - 1])
            p[i][j] = (-1, -1)
        
        if p[i][j] == (-1, 0):
            res1.append(seq1[i - 1])
            res2.append('-')
            i -= 1
        elif p[i][j] == (0, -1):
            res1.append('-')
            res2.append(seq2[j - 1])
            j -= 1
        elif p[i][j] == (-1, -1):
            if seq1[i - 1] != seq2[j - 1]:
                res1.append(seq1[i - 1].lower())
                res2.append(seq2[j - 1].lower())
            else:
                res1.append(seq1[i - 1])
                res2.append(seq2[j - 1])
            i, j = i - 1, j -1 
    res1.reverse()
    res2.reverse()
    print(''.join(res1))
    print(''.join(res2))
    return ''.join(res1), ''.join(res2)
            
import sys
from Bio import SeqIO

def main():
    sys.stdin = open("test_in.txt", "r")
    sys.stdout = open("test_out.txt", "w")
    input = sys.stdin.readline

    d_same, d_replace, d_open, d_extension = list(map(float, input().split()))
            
    sequences = [record.seq for record in SeqIO.parse(sys.stdin, "fasta")]
    seq1, seq2 = sequences[0], sequences[1]
    
    needleman_wunsch(d_same, d_replace, d_open, d_extension, seq1, seq2)
main()